# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.



In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 355 kB/s 
     |████████████████████████████████| 57 kB 2.9 MB/s 
     |████████████████████████████████| 582 kB 50.1 MB/s 
     |████████████████████████████████| 2.8 MB 58.0 MB/s 
     |████████████████████████████████| 596 kB 59.9 MB/s 
     |████████████████████████████████| 141 kB 45.7 MB/s 
     |████████████████████████████████| 419 kB 46.9 MB/s 
     |████████████████████████████████| 3.3 MB 42.4 MB/s 
     |████████████████████████████████| 880 kB 50.7 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 96 kB 6.3 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d362bce82fe00dec6e1cb84dfde2ea008ea9485998d495b2435f593ea183bf11
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Su

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    max_seq_length=128,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-doccls/epoch=0-val_loss=0.26.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [ ]:
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"

    return {
        'sentence': sentence,
        #'prediction': pred,
        'positive_data': f"긍정 {positive_prob * 100}" + "%",
        'negative_data': f"부정 {negative_prob * 100}" + "%",
        #'positive_width': f"{positive_prob * 100}%",
        #'negative_width': f"{negative_prob * 100}%",
    }

In [ ]:
while 1:
    sentence = input("문장을 입력하세요 : ")
    if sentence == "quit":  # 종료
        break
    print(inference_fn(sentence))
    print('\n')

문장을 입력하세요 : 코테이토 딥러닝 스터디 좋아
{'sentence': '코테이토 딥러닝 스터디 좋아', 'positive_data': '긍정 92.52%', 'negative_data': '부정 7.48%'}


문장을 입력하세요 : 공부 재미없다
{'sentence': '공부 재미없다', 'positive_data': '긍정 0.62%', 'negative_data': '부정 99.38%'}


문장을 입력하세요 : 저 배우 멋있다
{'sentence': '저 배우 멋있다', 'positive_data': '긍정 97.25%', 'negative_data': '부정 2.75%'}


문장을 입력하세요 : ㅇㅈ
{'sentence': 'ㅇㅈ', 'positive_data': '긍정 68.28999999999999%', 'negative_data': '부정 31.71%'}


문장을 입력하세요 : quit


# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [ ]:
!mkdir /root/.ngrok2 && echo "authtoken: 2CssDYIBD0AVFNE0D0o60rAt1nj_4rUKvHmHJ9Lfoi27mqaw2" > /root/.ngrok2/ngrok.yml

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://29c1-34-145-237-158.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


#정리

'데이터 로더’가 데이터 전처리를 하는 과정에서 배치 크기만큼 컬레이트를 해준 문장 인스턴스들을 뽑아내는 역할을 한다는 것을 알게 되었다.
이때 138p ClassificationDataset클래스가 데이터셋을 어떤 형태로 저장하고 있는지 새롭게 알게 되었다.(input_ids, attention_mask, token_type_ids, 라벨 정보) > 세그먼트 정보가 뭐지? 했는데 BERT 모델의 특징이란다. BERT모델에서 140p에 bert모델에서 프리트레인에 자주 쓰이는 과제인 ‘이어진 문서 맞히기’에서 첫 번째 문서를 0으로, 두 번째 문서를 1으로 id를 부여해서 0라벨 문서와 1라벨 문서를 학습해주는 것 같다.

141p에 배치 사이즈별로 학습된다고 하였는데, 141페이지에 인스턴스 1과 인스턴스2가 배치1로 들어가 학습되는 것을 보면 나중에 자신이 제공받는 혹은 사용할수 있는 데이터셋을 잘 분석해서 배치사이즈를 적절히 정해주는 것이 좋겠다는 생각이 들었다.(그런데 보통 배치사이즈는 32로 설정을 많이 하는데 왜 그런지 찾아보기)